<a href="https://colab.research.google.com/github/YaCpotato/python/blob/master/GBDTCompareWithIris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

from keras.utils import np_utils
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,classification_report
import time
from sklearn import model_selection
import pandas as pd

In [60]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data)
iris_df = iris_df.rename(columns={
    0: 'sepal_length',
    1: 'sepal_width',
    2: 'petal_length',
    3: 'petal_width'})
iris_df['target'] = iris.target
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [0]:
train_df, test_df = model_selection.train_test_split(iris_df, test_size=0.3) # 30%をテストに使う
train_df_y = train_df[['target']]
train_df_x = train_df.copy().drop('target', axis=1)
test_df_y = test_df[['target']]
test_df_x = test_df.copy().drop('target', axis=1)

In [62]:
import xgboost as xgb
# 今回はアヤメの種類を当てるクラス分類なのでXGBClassifier
clf = xgb.XGBClassifier()

# ハイパーパラメータ探索
clf_cv = model_selection.GridSearchCV(clf, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)
clf_cv.fit(train_df_x, [i[0] for i in train_df_y.values])
print(clf_cv.best_params_, clf_cv.best_score_)

# 良さげなパラメータで学習しなおす
clf = xgb.XGBClassifier(**clf_cv.best_params_)
clf.fit(train_df_x, [i[0] for i in train_df_y.values])

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'max_depth': 2, 'n_estimators': 50} 0.9238095238095237


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.1s finished


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=None, n_estimators=50, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [63]:
pred = clf.predict(test_df_x)
print(confusion_matrix([i[0] for i in test_df_y.values], pred))
print(classification_report([i[0] for i in test_df_y.values], pred))

[[15  0  0]
 [ 0 14  0]
 [ 0  1 15]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.93      1.00      0.97        14
           2       1.00      0.94      0.97        16

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

